In [1]:
import os
import subprocess
import psutil
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet50
from tqdm import tqdm
import mlflow
import mlflow.pytorch
from pynvml import (
    nvmlInit, nvmlDeviceGetHandleByIndex,
    nvmlDeviceGetUtilizationRates, nvmlDeviceGetMemoryInfo,
    nvmlDeviceGetTemperature, NVML_TEMPERATURE_GPU
)

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", DEVICE)

mlflow.set_experiment("ResNet50_MelAug")
try:
    mlflow.end_run()
except:
    pass
mlflow.start_run(log_system_metrics=True)

# log GPU info
gpu_info = next(
    (subprocess.run(cmd, capture_output=True, text=True).stdout
        for cmd in ["nvidia‑smi", "rocm‑smi"]
        if subprocess.run(f"command ‑v {cmd}", shell=True, capture_output=True).returncode == 0),
    "No GPU found."
)
mlflow.log_text(gpu_info, "gpu-info.txt")

nvmlInit()
gpu_handle = nvmlDeviceGetHandleByIndex(0)

def log_system_metrics_mlflow(step=None):
    mlflow.log_metric("system.cpu.utilization", psutil.cpu_percent(), step=step)
    mem = psutil.virtual_memory()
    mlflow.log_metric("system.memory.used", mem.used, step=step)
    mlflow.log_metric("system.memory.percent", mem.percent, step=step)
    gpu_util = nvmlDeviceGetUtilizationRates(gpu_handle).gpu
    mlflow.log_metric("system.gpu.0.utilization", gpu_util, step=step)
    gpu_mem = nvmlDeviceGetMemoryInfo(gpu_handle)
    mlflow.log_metric("system.gpu.0.memory.used", gpu_mem.used, step=step)
    mlflow.log_metric("system.gpu.0.memory.percent",
                      (gpu_mem.used / gpu_mem.total) * 100, step=step)
    gpu_temp = nvmlDeviceGetTemperature(gpu_handle, NVML_TEMPERATURE_GPU)
    mlflow.log_metric("system.gpu.0.temperature", gpu_temp, step=step)

2025/05/07 02:23:34 INFO mlflow.tracking.fluent: Experiment with name 'ResNet50_MelAug' does not exist. Creating a new experiment.


Using device: cuda


2025/05/07 02:23:34 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


In [3]:
tax_df      = pd.read_csv("/home/jovyan/Data/birdclef-2025/taxonomy.csv")
classes     = sorted(tax_df['primary_label'].astype(str).tolist())
num_classes = len(classes)


In [4]:
class MelAugDataset(Dataset):
    def __init__(self, manifest_csv: str, metadata_csv: str,
                 feature_base: str, classes: list, mel_key="mel"):
        """
        manifest_csv: path to manifest_train.csv or manifest_test.csv
        metadata_csv: path to train.csv for secondary_labels
        feature_base: root path of Features, e.g. "/home/jovyan/Features"
        classes: global sorted list of all primary_label codes
        """
        m_df = pd.read_csv(manifest_csv)
        # build full path under Features/mel_aug/
        m_df["mel_path"] = (
            m_df["mel_aug_path"].astype(str)
                .str.lstrip(os.sep)
                .apply(lambda p: os.path.join(feature_base, "mel_aug", p))
        )

        # load secondary_labels
        meta = pd.read_csv(metadata_csv, usecols=["filename","secondary_labels"])
        meta["recording_id"]   = meta.filename.str.replace(r"\.ogg$", "", regex=True)
        meta["sec_list"]       = meta.secondary_labels.fillna("").str.split()
        sec_map = dict(zip(meta.recording_id, meta.sec_list))

        self.rows = []
        self.classes     = classes
        self.label2idx   = {lab:i for i, lab in enumerate(self.classes)}
        self.num_classes = len(self.classes)
        self.mel_key     = mel_key

        for _, row in tqdm(m_df.iterrows(),
                          total=len(m_df),
                          desc=f"Building {os.path.basename(manifest_csv)}"):
            rid  = row.chunk_id.split("_chk")[0]
            labs = [row.primary_label] + sec_map.get(rid, [])
            # filter out any labels not in taxonomy
            labs = [l for l in labs if l in self.label2idx]
            self.rows.append({
                "mel_path": row.mel_path,
                "labels":   labs
            })

    def __len__(self):
        return len(self.rows)

    def __getitem__(self, idx):
        r   = self.rows[idx]
        npz = np.load(r["mel_path"])
        arr = npz[self.mel_key]             # (n_mels, n_frames)
        x   = torch.from_numpy(arr).unsqueeze(0).float()  # (1, n_mels, n_frames)
        y   = torch.zeros(self.num_classes, dtype=torch.float32)
        for lab in r["labels"]:
            y[self.label2idx[lab]] = 1.0
        return x, y

In [7]:
def get_resnet50_multilabel(num_classes: int):
    model = resnet50(weights=None)
    # adapt stem conv to 1‑channel
    model.conv1 = nn.Conv2d(
        in_channels=1,
        out_channels=model.conv1.out_channels,
        kernel_size=model.conv1.kernel_size,
        stride=model.conv1.stride,
        padding=model.conv1.padding,
        bias=False
    )
    # replace final FC
    in_feat = model.fc.in_features
    model.fc = nn.Linear(in_feat, num_classes)
    return model

In [8]:
FEATURE_BASE   = "/home/jovyan/Features"
TRAIN_MANIFEST = os.path.join(FEATURE_BASE, "manifest_train.csv")
TEST_MANIFEST  = os.path.join(FEATURE_BASE, "manifest_test.csv")
TRAIN_CSV      = "/home/jovyan/Data/birdclef-2025/train.csv"

BATCH_SIZE = 16
LR         = 1e-4
EPOCHS     = 1

train_ds = MelAugDataset(TRAIN_MANIFEST, TRAIN_CSV, FEATURE_BASE, classes)
test_ds  = MelAugDataset(TEST_MANIFEST,  TRAIN_CSV, FEATURE_BASE, classes)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                          shuffle=True,  num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE,
                          shuffle=False, num_workers=4, pin_memory=True)

model     = get_resnet50_multilabel(num_classes).to(DEVICE)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

mlflow.log_params({
    "model":        "resnet50_scratch",
    "input":        "mel_aug",
    "num_classes":  num_classes,
    "batch_size":   BATCH_SIZE,
    "lr":           LR,
    "epochs":       EPOCHS
})

Building manifest_test.csv: 100%|██████████| 11022/11022 [00:00<00:00, 24107.85it/s]


In [9]:
best_test_acc, best_ckpt = 0.0, None

for epoch in range(1, EPOCHS+1):
    # — Train —
    model.train()
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch} Train", unit="batch")
    run_loss, correct, total = 0.0, 0, 0

    for xb, yb in train_bar:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        optimizer.zero_grad()
        logits = model(xb)
        loss   = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        run_loss += loss.item() * xb.size(0)
        preds    = (torch.sigmoid(logits) > 0.5).float()
        correct  += (preds == yb).all(dim=1).sum().item()
        total    += xb.size(0)

        train_bar.set_postfix({
            "loss": f"{run_loss/total:.4f}",
            "acc":  f"{correct/total:.4f}"
        })

    train_loss = run_loss / total
    train_acc  = correct / total

    # — Test —
    model.eval()
    test_bar = tqdm(test_loader, desc=f"Epoch {epoch} Test ", unit="batch")
    test_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for xb, yb in test_bar:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            logits = model(xb)
            loss   = criterion(logits, yb)

            test_loss += loss.item() * xb.size(0)
            preds     = (torch.sigmoid(logits) > 0.5).float()
            correct   += (preds == yb).all(dim=1).sum().item()
            total     += xb.size(0)

            test_bar.set_postfix({
                "loss": f"{test_loss/total:.4f}",
                "acc":  f"{correct/total:.4f}"
            })

    test_loss /= total
    test_acc   = correct / total

    # checkpoint
    ckpt = f"resnet50_epoch_{epoch}.pt"
    torch.save({
        "epoch":       epoch,
        "model_state": model.state_dict(),
        "optim_state": optimizer.state_dict(),
        "train_loss":  train_loss,
        "test_loss":   test_loss
    }, ckpt)

    mlflow.log_metrics({
        "train_loss":    train_loss,
        "train_accuracy": train_acc,
        "test_loss":     test_loss,
        "test_accuracy": test_acc
    }, step=epoch)
    log_system_metrics_mlflow(step=epoch)
    mlflow.log_artifact(ckpt, artifact_path="checkpoints")

    if test_acc > best_test_acc:
        best_test_acc, best_ckpt = test_acc, ckpt

    print(f"→ Epoch {epoch}/{EPOCHS}  "
          f"Train loss={train_loss:.4f}, acc={train_acc:.4f} │ "
          f"Test loss={test_loss:.4f}, acc={test_acc:.4f}")

Epoch 1 Test : 100%|██████████| 689/689 [00:07<00:00, 92.76batch/s, loss=0.0170, acc=0.3139]


→ Epoch 1/1  Train loss=0.0194, acc=0.1991 │ Test loss=0.0170, acc=0.3139


In [10]:
mlflow.log_metric("best_test_accuracy", best_test_acc)
mlflow.log_artifact(best_ckpt, artifact_path="model")
mlflow.end_run()

2025/05/07 02:31:57 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/05/07 02:31:57 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


🏃 View run crawling-quail-65 at: http://192.5.87.49:8000/#/experiments/2/runs/ffd8bef648eb4cbc9eca50b8b4842c89
🧪 View experiment at: http://192.5.87.49:8000/#/experiments/2
